# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

import nltk
nltk.download(['punkt','wordnet','stopwords','words'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [20]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessageToCategories',con=engine)
X = df['message'].values 
Y = df.iloc[:,4:]
#X = df.message.values
#Y = df[df.columns[4:]].values

In [21]:
classes = Y.columns
classes

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'], dtype=object)

In [5]:
Y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
len(X)

26216

In [7]:
len(Y)

26216

### 2. Write a tokenization function to process your text data

In [8]:
text = re.sub(r"[^a-zA-Z]"," ",text).lower()
text

TypeError: cannot use a string pattern on a bytes-like object

In [24]:
def tokenize(text):
    #wrds = set(words.words())
    text = re.sub(r"[^a-zA-Z]"," ",text).lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    #tokens = [t for t in tokens if t in wrds]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [ ]:
'''words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())'''

In [25]:
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'could', 'pas'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'hospital', 'st', 'need', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['west', 'side', 'rest', 'country', 'today', 'tonight'] 

Information about the National Palace-
['information', 'national', 'palace'] 

Storm at sacred heart of jesus
['storm', 'sacred', 'heart'] 

Please, we need tents and water. We are in Silo, Thank you!
['please', 'need', 'water', 'silo', 'thank'] 

I would like to receive the messages, thank you
['would', 'like', 'receive', 'thank'] 

I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )
['health', 'area'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [27]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                ])

'''pipeline = Pipeline([
               ('vect', CountVectorizer(ngram_range=(1,3), max_df=0.50 ) ),
               ('tfidf', TfidfTransformer() ),
               ('clf', OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                          alpha=1e-4, n_iter=5, random_state=42,
                                          shuffle=True, n_jobs=-1) )),
                ])'''

"pipeline = Pipeline([\n               ('vect', CountVectorizer(ngram_range=(1,3), max_df=0.50 ) ),\n               ('tfidf', TfidfTransformer() ),\n               ('clf', OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',\n                                          alpha=1e-4, n_iter=5, random_state=42,\n                                          shuffle=True, n_jobs=-1) )),\n                ])"

In [16]:
#vect = CountVectorizer(tokenizer=tokenize)
#X_train = vect.fit_transform(X_train)

In [17]:
#dict = vect.vocabulary_
#sorted(dict.items(), key=lambda kv: kv[1], reverse=True)

[('zzz', 26381),
 ('zz', 26380),
 ('zy', 26379),
 ('zxlo', 26378),
 ('zxje', 26377),
 ('zxcap', 26376),
 ('zwkmjkkz', 26375),
 ('zwaar', 26374),
 ('zvun', 26373),
 ('zurich', 26372),
 ('zuqj', 26371),
 ('zung', 26370),
 ('zugdidi', 26369),
 ('zuevo', 26368),
 ('zud', 26367),
 ('ztfbqjht', 26366),
 ('zrcs', 26365),
 ('zqgrb', 26364),
 ('zpst', 26363),
 ('zota', 26362),
 ('zordon', 26361),
 ('zoonotic', 26360),
 ('zoonosis', 26359),
 ('zoomed', 26358),
 ('zoom', 26357),
 ('zood', 26356),
 ('zoo', 26355),
 ('zonne', 26354),
 ('zongo', 26353),
 ('zoneb', 26352),
 ('zonea', 26351),
 ('zone', 26350),
 ('zonal', 26349),
 ('zona', 26348),
 ('zon', 26347),
 ('zomgggg', 26346),
 ('zombie', 26345),
 ('zoktr', 26344),
 ('zodiac', 26343),
 ('zobas', 26342),
 ('zo', 26341),
 ('zn', 26340),
 ('zmyilk', 26339),
 ('zmsfjawh', 26338),
 ('zmir', 26337),
 ('zm', 26336),
 ('zlxvo', 26335),
 ('zlu', 26334),
 ('zll', 26333),
 ('zj', 26332),
 ('ziziphus', 26331),
 ('zixi', 26330),
 ('zitig', 26329),
 ('ziti',

In [42]:
'''sorted(dict.items())
new_list = {}
words = set(nltk.corpus.words.words())
for (key,value) in dict.items():
    if key in words:
        new_list.update([(key,value)])
len(new_list)'''

11295

In [35]:
#dict

{'upsurge': 24740,
 'cholera': 3993,
 'case': 3474,
 'attributed': 1564,
 'lack': 12940,
 'clean': 4160,
 'water': 25428,
 'proper': 18399,
 'sanitation': 20485,
 'facility': 7990,
 'coupled': 5066,
 'current': 5342,
 'heavy': 10208,
 'rain': 18861,
 'high': 10336,
 'population': 17908,
 'movement': 15206,
 'neighboring': 15696,
 'country': 5055,
 'one': 16460,
 'coast': 4268,
 'guard': 9703,
 'ship': 21214,
 'carrying': 3456,
 'search': 20747,
 'rescue': 19616,
 'quilon': 18760,
 'province': 18465,
 'turn': 24179,
 'worked': 25838,
 'raise': 18882,
 'awareness': 1686,
 'among': 877,
 'industrial': 11259,
 'sector': 20789,
 'need': 15660,
 'affected': 419,
 'assisting': 1464,
 'also': 781,
 'important': 11095,
 'security': 20798,
 'reform': 19272,
 'including': 11178,
 'reconstitution': 19172,
 'new': 15758,
 'afghan': 436,
 'army': 1321,
 'police': 17842,
 'justice': 12227,
 'third': 23432,
 'year': 26098,
 'computing': 4566,
 'science': 20667,
 'university': 24574,
 'ruben': 20124,
 

In [18]:
#sorted(dict.items(), key=lambda kv: kv[1])

[('aa', 0),
 ('aaa', 1),
 ('aaaaaaaa', 2),
 ('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
  3),
 ('aab', 4),
 ('aabad', 5),
 ('aaca', 6),
 ('aacf', 7),
 ('aadtth', 8),
 ('aah', 9),
 ('aahar', 10),
 ('aai', 11),
 ('aaj', 12),
 ('aalst', 13),
 ('aap', 14),
 ('aar', 15),
 ('aaron', 16),
 ('aarqlt', 17),
 ('aarthik', 18),
 ('ab', 19),
 ('aba', 20),
 ('ababa', 21),
 ('abad', 22),
 ('abadan', 23),
 ('abala', 24),
 ('abandon', 25),
 ('abandoned', 26),
 ('abandoning', 27),
 ('abandonned', 28),
 ('abaout', 29),
 ('abarho', 30),
 ('abated', 31),
 ('abattre', 32),
 ('abbas', 33),
 ('abbotabad', 34),
 ('abbottabad', 35),
 ('abbyloves', 36),
 ('abbymaroon', 37),
 ('abc', 38),
 ('abcb', 39),
 ('abdi', 40),
 ('abdimalik', 41),
 ('abdirahman', 42),
 ('abdomen', 43),
 ('abdominal', 44),
 ('abdoul', 45),
 ('abdoulaye', 46),
 ('abdrahmanov', 47),
 ('abducted', 48),
 ('abduction', 49),
 (

In [60]:
#tfidf = TfidfTransformer()
#X_train_tfidf = tfidf.fit_transform(X_train)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

KeyboardInterrupt: 

In [88]:
#clf = RandomForestClassifier()
#model = MultiOutputClassifier(clf)
#model.fit(X_train_tfidf,Y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
from sklearn.metrics import classification_report, accuracy_score

In [16]:
Y_pred = pipeline.predict(X_test)

In [89]:
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)
#Y_pred = model.predict(X_test_tfidf)

In [ ]:
multi_f1 = multioutput_fscore(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score (custom definition) {0:.2f}%\n'.format(multi_f1*100))

In [17]:
#print(classification_report(Y_test[1],Y_pred[1],labels=36, target_names=classes))
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=classes))

                        precision    recall  f1-score   support

               related       0.79      0.44      0.57      1105
               request       0.00      0.00      0.00        24
                 offer       0.75      0.60      0.67      2710
           aid_related       0.48      0.10      0.16       506
          medical_help       0.79      0.18      0.29       306
      medical_products       0.65      0.09      0.15       171
     search_and_rescue       0.00      0.00      0.00       151
              security       0.74      0.07      0.13       237
              military       0.00      0.00      0.00         0
           child_alone       0.86      0.44      0.59       405
                 water       0.83      0.52      0.64       711
                  food       0.78      0.35      0.48       561
               shelter       1.00      0.14      0.25       104
              clothing       0.67      0.04      0.08       144
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
print("         Accuracy Score: ")
print("         ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))

         Accuracy Score: 
         ---------------
                  related : 0.80
                  request : 0.89
                    offer : 1.00
              aid_related : 0.75
             medical_help : 0.92
         medical_products : 0.96
        search_and_rescue : 0.97
                 security : 0.98
                 military : 0.97
              child_alone : 1.00
                    water : 0.96
                     food : 0.94
                  shelter : 0.94
                 clothing : 0.99
                    money : 0.98
           missing_people : 0.99
                 refugees : 0.97
                    death : 0.96
                other_aid : 0.87
   infrastructure_related : 0.93
                transport : 0.96
                buildings : 0.96
              electricity : 0.98
                    tools : 0.99
                hospitals : 0.99
                    shops : 1.00
              aid_centers : 0.99
     other_infrastructure : 0.95
          weather_related

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
parameters = {'clf__estimator__n_estimators':[10,20],
              'clf__estimator__min_samples_split':[2,4]
              }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv

param_grid = dict(reduce_dim=[None, PCA(5), PCA(10)],
...                   clf=[SVC(), LogisticRegression()],
...                   clf__C=[0.1, 10, 100])
>>> grid_search = GridSearchCV(pipe, param_grid=param_grid)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
Y_pred = cv.predict(X_test)

In [28]:
print(classification_report(Y_test.iloc[:,1:].values, np.array([x[1:] for x in Y_pred]), target_names=classes))
print("         Accuracy Score: ")
print("         ---------------")
for i in range(Y_test.shape[1]):
    print('%25s : %.2f' %(classes[i], accuracy_score(Y_test.values[:,i], Y_pred[:,i])))

                        precision    recall  f1-score   support

               related       0.85      0.43      0.57      1105
               request       0.00      0.00      0.00        25
                 offer       0.76      0.57      0.65      2653
           aid_related       0.64      0.09      0.15       494
          medical_help       0.74      0.05      0.10       325
      medical_products       0.75      0.05      0.09       196
     search_and_rescue       0.00      0.00      0.00       122
              security       0.67      0.07      0.12       213
              military       0.00      0.00      0.00         0
           child_alone       0.89      0.37      0.52       426
                 water       0.86      0.43      0.57       679
                  food       0.83      0.25      0.38       571
               shelter       0.75      0.10      0.18        86
              clothing       1.00      0.03      0.06       162
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.